In [1]:
library('SelectSim')
library('tidyverse')
library('effsize')
library('ggpubr')
library('ggsci')
library('qvalue')

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] qvalue_2.34.0     ggsci_3.0.0       ggpubr_0.6.0      effsize_0.8.1    
 [5] lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1     dplyr_1.1.4      
 [9] purrr_1.0.2       readr_2.1.5       tidyr_1.3.0       tibble_3.2.1     

# TCGA Pan cancer

In [3]:
run_data <- readRDS('/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_tcga_run_data.rds')

In [4]:
str(run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:9082] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  .. ..$ truncating: num [1:396, 1:9082] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	9082 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  .. .. ..$ mutation: num [1:9082] 56 47 40 422 214 588 14 8 19 8 ...
  .. ..$ truncating:'data.frame':	9082 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-

In [5]:
gam<-run_data$M$M$missense+run_data$M$M$truncating
gam[gam>1]<-1

In [6]:
all_tmb <- run_data$M$tmb$missense
all_tmb$mutation<-all_tmb$mutation+ run_data$M$tmb$truncating$mutation

In [8]:
catalougue_result <- readRDS('/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/results/all_merged_results_v15.rds')

In [9]:
tcga_selectsim<- catalougue_result %>% filter(Tumor_run=='Pan-can' & Cohort=='TCGA') %>% mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),
                                                  SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - ")))

In [10]:
dim(tcga_selectsim)

[1] 52326    27

In [11]:
tcga_select <- readRDS('/mnt/ndata/arvind/co_mutation_project/analysis/robustness_benchmarking/results/tcga_select_all_results.rds')

In [12]:
head(gam)

,TCGA-02-0047-01,TCGA-02-0055-01,TCGA-AB-2805-03,TCGA-AB-2806-03,TCGA-AB-2807-03,TCGA-AB-2808-03,TCGA-AB-2810-03,TCGA-AB-2811-03,TCGA-AB-2813-03,TCGA-AB-2814-03,⋯,TCGA-ZP-A9CY-01,TCGA-ZP-A9CZ-01,TCGA-ZP-A9D0-01,TCGA-ZP-A9D1-01,TCGA-ZQ-A9CR-01,TCGA-ZR-A9CJ-01,TCGA-ZS-A9CD-01,TCGA-ZS-A9CF-01,TCGA-ZT-A8OM-01,TCGA-ZX-AA5X-01
AKT1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ALK,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
APC,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AR,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ATM,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
BAP1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [13]:
compute_cohens_d <- function(result_df,gam,tmb_df){
    without_log <-list()
    with_log<-list()
    pairs<-list()
    for(i in c(1:nrow(result_df))){
        co_mut <- result_df[i,'overlap']
        gene1<-result_df[i,'SFE_1']
        gene2<-result_df[i,'SFE_2']
        pairs[[i]] <-result_df[i,'id']
        wild_type <- (names(which(colSums(gam[c(gene1,gene2),])==0)))
        co_mutated <- (names(which(colSums(gam[c(gene1,gene2),])==2)))
        mutated_df<-tmb_df %>% mutate(status=case_when(sample %in% co_mutated ~ 1,
                                    sample %in% wild_type ~ 0)) %>% filter(status %in% c(0,1))

        group1 <- ((mutated_df %>% filter(sample %in% co_mutated))$mutation)
        group2 <- ((mutated_df %>% filter(sample %in% wild_type))$mutation)
        cohen_d <- cohen.d(group1, group2)$estimate
        without_log[[i]]<-cohen_d
        group1 <- log10(1+(mutated_df %>% filter(sample %in% co_mutated))$mutation)
        group2 <- log10(1+(mutated_df %>% filter(sample %in% wild_type))$mutation)
        cohen_d <- cohen.d(group1, group2)$estimate
        with_log[[i]]<-cohen_d
    }
   return(list(pairs,with_log,without_log))
}

##  SelectSim

In [14]:
co_mut_df <- tcga_selectsim %>% filter(name %in% tcga_selectsim$name) %>% filter(FDR==TRUE) %>% filter(type=='CO') %>% mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),
                                                  SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))) %>% mutate(id_type=paste(id,type,sep="_")) %>% filter(overlap>=5)

In [15]:
select_sim_result <- compute_cohens_d(result_df = co_mut_df,gam = gam,tmb_df =all_tmb )

In [16]:
select_sim_result[[1]]

[[1]]
[1] "CDKN2A - TP53"

[[2]]
[1] "ATRX - IDH1"

[[3]]
[1] "ARID1A - PTEN"

[[4]]
[1] "CTNNB1 - PTEN"

[[5]]
[1] "PIK3R1 - PTEN"

[[6]]
[1] "RB1 - TP53"

[[7]]
[1] "CDH1 - PIK3CA"

[[8]]
[1] "PIK3CA - PTEN"

[[9]]
[1] "MAP3K1 - PIK3CA"

[[10]]
[1] "ATRX - TP53"

[[11]]
[1] "CTCF - PTEN"

[[12]]
[1] "ARID1A - PIK3CA"

[[13]]
[1] "KMT2C - KMT2D"

[[14]]
[1] "CIC - IDH1"

[[15]]
[1] "KRAS - SMAD4"

[[16]]
[1] "CDKN2A - KRAS"

[[17]]
[1] "KRAS - PTEN"

[[18]]
[1] "ARID1A - CTNNB1"

[[19]]
[1] "ARID1A - CTCF"

[[20]]
[1] "FLT3 - NPM1"

[[21]]
[1] "CTNNB1 - PIK3CA"

[[22]]
[1] "ARID5B - PTEN"

[[23]]
[1] "PBRM1 - SETD2"

[[24]]
[1] "PTEN - ZFHX3"

[[25]]
[1] "KRAS - STK11"

[[26]]
[1] "FGFR2 - PTEN"

[[27]]
[1] "DNMT3A - NPM1"

[[28]]
[1] "ARID1A - CCND1"

[[29]]
[1] "PTEN - RB1"

[[30]]
[1] "KMT2C - PIK3CA"

[[31]]
[1] "KRAS - RNF43"

[[32]]
[1] "CTCF - KMT2B"

[[33]]
[1] "CTCF - PIK3CA"

[[34]]
[1] "KRAS - RBM10"

[[35]]
[1] "PPP2R1A - TP53"

[[36]]
[1] "CCND1 - PTEN"

[[37]]
[1] "ARID1A - KRAS"

[[38]]
[1] "GNAQ - SF3B1"

[[39]]
[1] "INPPL1 - PTEN"

[[40]]
[1] "CDH1 - ERBB2"

[[41]]
[1] "PIK3CA - RUNX1"

[[42]]
[1] "JAK1 - KRAS"

[[43]]
[1] "ARID5B - PIK3CA"

[[44]]
[1] "KMT2B - PTEN"

[[45]]
[1] "PBRM1 - VHL"

[[46]]
[1] "ARID1A - JAK1"

[[47]]
[1] "PTEN - RNF43"

[[48]]
[1] "FGFR3 - STAG2"

[[49]]
[1] "CTCF - CTNNB1"

[[50]]
[1] "ARID1A - ARID1B"

[[51]]
[1] "AKT1 - GATA3"

[[52]]
[1] "FUBP1 - IDH1"

[[53]]
[1] "MAP2K4 - PIK3CA"

[[54]]
[1] "KDM6A - STAG2"

[[55]]
[1] "KMT2B - KMT2D"

[[56]]
[1] "KMT2B - RNF43"

[[57]]
[1] "FGFR3 - KDM6A"

[[58]]
[1] "NSD1 - PTEN"

[[59]]
[1] "AMER1 - SMAD4"

[[60]]
[1] "KEAP1 - STK11"

[[61]]
[1] "CDH1 - TBX3"

[[62]]
[1] "ARID1A - NSD1"

[[63]]
[1] "DNMT3A - FLT3"

[[64]]
[1] "NCOR1 - PIK3CA"

[[65]]
[1] "KMT2B - KRAS"

[[66]]
[1] "KMT2C - NFE2L2"

[[67]]
[1] "ARID1B - GLI1"

[[68]]
[1] "FAT1 - HRAS"

[[69]]
[1] "EP300 - RB1"

[[70]]
[1] "PIK3R1 - ZFHX3"

[[71]]
[1] "BCORL1 - KMT2D"

[[72]]
[1] "CASP8 - FAT1"

[[73]]
[1] "CASP8 - HLA-A"

[[74]]
[1] "GPS2 - PIK3CA"

[[75]]
[1] "CASP8 - HRAS"

[[76]]
[1] "CCND1 - PIK3CA"

[[77]]
[1] "AJUBA - TP53"

[[78]]
[1] "CCND1 - KMT2B"

##  Select

In [17]:
co_mut_df <- tcga_select %>% filter(name %in% tcga_selectsim$name) %>% filter(wMI_p.value_FDR==TRUE) %>% filter(direction=='CO') %>% mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),
                                                  SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))) %>% filter(overlap>=5)

In [18]:
dim(co_mut_df)

[1] 78 27

In [19]:
select_result <- compute_cohens_d(result_df = co_mut_df,gam = gam,tmb_df =all_tmb )

dim(tcga_select)

In [21]:
dim(tcga_selectsim)

[1] 52326    27

In [22]:
 tcga_select %>% filter(name %in% tcga_selectsim$name) %>% filter(wMI_p.value_FDR) %>% count(direction)

direction,n
<chr>,<int>
CO,110
ME,78


In [23]:
length(select_result[[1]])

[1] 78

## Combine results

In [26]:
select_plot_df <- data.frame('pairs'=unlist(select_result[[1]]),'effect_size'=c(unlist(select_result[[2]])),'category'=c(rep('log',78)))
selectsim_plot_df <- data.frame('pairs'=unlist(select_sim_result[[1]]),'effect_size'=c(unlist(select_sim_result[[2]])),'category'=c(rep('log',78)))

In [27]:
head(select_plot_df)

,pairs,effect_size,category
,<chr>,<dbl>,<chr>
1,AR - ERBB4,3.4739465,log
2,AR - PTCH1,3.1338454,log
3,BRCA2 - TSC1,2.6383566,log
4,CDKN2A - TP53,0.8616649,log
5,CREBBP - POLE,3.2690304,log
6,DNMT3A - NPM1,-1.2372010,log


In [28]:
all_plot_df <- rbind(select_plot_df %>% mutate('algo'='Select'),
                     selectsim_plot_df %>% mutate('algo'='SelectSim'))

In [29]:
head(all_plot_df)

,pairs,effect_size,category,algo
,<chr>,<dbl>,<chr>,<chr>
1,AR - ERBB4,3.4739465,log,Select
2,AR - PTCH1,3.1338454,log,Select
3,BRCA2 - TSC1,2.6383566,log,Select
4,CDKN2A - TP53,0.8616649,log,Select
5,CREBBP - POLE,3.2690304,log,Select
6,DNMT3A - NPM1,-1.2372010,log,Select


In [30]:
plot_df <- all_plot_df  %>% select(pairs,effect_size,algo) %>% distinct()

In [31]:
head(plot_df)

,pairs,effect_size,algo
,<chr>,<dbl>,<chr>
1,AR - ERBB4,3.4739465,Select
2,AR - PTCH1,3.1338454,Select
3,BRCA2 - TSC1,2.6383566,Select
4,CDKN2A - TP53,0.8616649,Select
5,CREBBP - POLE,3.2690304,Select
6,DNMT3A - NPM1,-1.2372010,Select


In [39]:
saveRDS(plot_df,file='/mnt/ndata/arvind/co_mutation_project/analysis/robustness_benchmarking/figures/tcga_mutational_load_plot_data.rds')

# LUAD

In [41]:
dim(gam)

[1]  396 9082

In [42]:
luad_gam<-gam[,which(run_data$sample.class=='LUAD')]

In [43]:
luad_selectsim <- readRDS('/mnt/ndata/arvind/co_mutation_project/analysis/robustness_benchmarking/results/LUAD/selectX/tcga_LUAD.rds')

In [44]:
luad_select <- readRDS('/mnt/ndata/arvind/co_mutation_project/analysis/robustness_benchmarking/results/LUAD/select/luad_all_results.rds')

In [45]:
luad_gamtoc <- read.table('/mnt/ndata/arvind/co_mutation_project/analysis/robustness_benchmarking/results/LUAD/gamtoc/luad_all.txt',header = TRUE)

In [46]:
luad_wesmeco <- read.table('/mnt/ndata/arvind/co_mutation_project/analysis/robustness_benchmarking/results/LUAD/wesme/LUAD_smut_co_pvs_0.1.txt',sep = "\t",header = TRUE)

In [47]:
luad_coocur <- readRDS('/mnt/ndata/arvind/co_mutation_project/analysis/robustness_benchmarking/results/LUAD/cooccur_luad_all_result.rds')

In [48]:
compute_cohens_d <- function(result_df,gam,tmb_df){
    without_log <-list()
    with_log<-list()
    pairs<-list()
    k=1
    for(i in c(1:nrow(result_df))){
        #co_mut <- result_df[i,'overlap']
        gene1<-result_df[i,'SFE_1']
        gene2<-result_df[i,'SFE_2']
        
        wild_type <- (names(which(colSums(gam[c(gene1,gene2),])==0)))
        co_mutated <- (names(which(colSums(gam[c(gene1,gene2),])==2)))
        if(length(co_mutated)>=2){
            mutated_df<-tmb_df %>% mutate(status=case_when(sample %in% co_mutated ~ 1,
                                        sample %in% wild_type ~ 0)) %>% filter(status %in% c(0,1))
    
            group1 <- ((mutated_df %>% filter(sample %in% co_mutated))$mutation)
            group2 <- ((mutated_df %>% filter(sample %in% wild_type))$mutation)
            cohen_d <- cohen.d(group1, group2)$estimate
            without_log[[k]]<-cohen_d
            group1 <- log10(1+(mutated_df %>% filter(sample %in% co_mutated))$mutation)
            group2 <- log10(1+(mutated_df %>% filter(sample %in% wild_type))$mutation)
            cohen_d <- cohen.d(group1, group2)$estimate
            with_log[[k]]<-cohen_d   
            pairs[[k]] <-result_df[i,'id']
            k=k+1
        }
    }
   return(list(pairs,with_log,without_log))
}

In [49]:
select_sim_result <- compute_cohens_d(result_df = luad_selectsim %>% filter(FDR==TRUE) %>% filter(type=='CO') %>% 
                                  mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))),gam = luad_gam,tmb_df =all_tmb )

In [50]:
select_result <- compute_cohens_d(result_df = luad_select %>% filter(wMI_p.value_FDR) %>% 
                                  mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))) %>% filter(direction=='CO')
                                  ,gam = luad_gam,tmb_df =all_tmb )

In [51]:
gamtoc_result <- compute_cohens_d(result_df = luad_gamtoc %>% filter(corr>=0) %>% mutate(SFE_1=node1) %>% mutate(SFE_2=node2) %>% 
                                  mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))) 
                                  ,gam = luad_gam,tmb_df =all_tmb )

In [52]:
wesme_result <- compute_cohens_d(result_df = luad_wesmeco %>% filter(`pv..ws.`<=0.1) %>% mutate(SFE_1=gene1) %>% mutate(SFE_2=gene2) %>% 
                                  mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))) 
                                  ,gam = luad_gam,tmb_df =all_tmb )

In [53]:
cooccur_result <- compute_cohens_d(result_df = luad_coocur %>% filter(`p_gt`<=0.05) %>% 
                                  mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))) 
                                  ,gam = luad_gam,tmb_df =all_tmb )

In [55]:
select_plot_df <- data.frame('pairs'=unlist(select_result[[1]]),'effect_size'=c(unlist(select_result[[2]])),'category'=c(rep('log',5)))

In [56]:
selectsim_plot_df <- data.frame('pairs'=unlist(select_sim_result[[1]]),'effect_size'=c(unlist(select_sim_result[[2]])),'category'=c(rep('log',13)))

In [57]:
gamtoc_plot_df <- data.frame('pairs'=unlist(gamtoc_result[[1]]),'effect_size'=c(unlist(gamtoc_result[[2]])),'category'=c(rep('log',7)))

In [58]:
wesme_plot_df <- data.frame('pairs'=unlist(wesme_result[[1]]),'effect_size'=c(unlist(wesme_result[[2]])),'category'=c(rep('log',65)))

In [59]:
coocur_plot_df <- data.frame('pairs'=unlist(cooccur_result[[1]]),'effect_size'=c(unlist(cooccur_result[[2]])),'category'=c(rep('log',14)))

In [60]:
head(select_plot_df)

,pairs,effect_size,category
,<chr>,<dbl>,<chr>
1,ARID1A - CARD11,0.6380519,log
2,KDR - SLFN11,1.4144201,log
3,BRCA1 - KDR,0.8699578,log
4,PBRM1 - SETBP1,1.0518664,log
5,ATRX - VAV1,1.6653571,log


In [61]:
head(selectsim_plot_df)

,pairs,effect_size,category
,<chr>,<dbl>,<chr>
1,KRAS - STK11,0.0003633509,log
2,EGFR - TP53,-0.3048752548,log
3,KEAP1 - STK11,0.0347003282,log
4,KRAS - RBM10,0.0578319825,log
5,RB1 - TP53,0.4468832872,log
6,NF1 - TP53,0.7069614431,log


In [62]:
all_plot_df <- rbind(select_plot_df %>% mutate('algo'='Select'),
                     selectsim_plot_df %>% mutate('algo'='SelectSim'),
                    gamtoc_plot_df %>% mutate('algo'='GamToc'),
                    wesme_plot_df %>% mutate('algo'='WesME/CO'),
                    coocur_plot_df %>% mutate('algo'='cooccur'))

In [63]:
head(all_plot_df)

,pairs,effect_size,category,algo
,<chr>,<dbl>,<chr>,<chr>
1,ARID1A - CARD11,0.6380518979,log,Select
2,KDR - SLFN11,1.4144200841,log,Select
3,BRCA1 - KDR,0.8699578453,log,Select
4,PBRM1 - SETBP1,1.0518663974,log,Select
5,ATRX - VAV1,1.6653570605,log,Select
6,KRAS - STK11,0.0003633509,log,SelectSim


In [64]:
a<-(all_plot_df %>% filter(algo=='SelectSim'))$pairs
b<-(all_plot_df %>% filter(algo=='Select'))$pairs
c<-(all_plot_df %>% filter(algo=='GamToc'))$pairs
d<-(all_plot_df %>% filter(algo=='WesME/CO'))$pairs
e<-(all_plot_df %>% filter(algo=='cooccur'))$pairs
#f<-(all_plot_df %>% filter(algo=='coselens'))$pairs

In [66]:
common <- Reduce(intersect,list(a,c,d,e))

In [67]:
selectsim_only <- setdiff(a,c(b,c,d))
select_only <- setdiff(b,c(a,c,d))
gamtoc_only <- setdiff(c,c(a,b,d))
WesME_only <- setdiff(d,c(a,b,c))

In [69]:
all_plot_df<-all_plot_df %>% mutate(alog_cat = case_when(pairs %in% setdiff(c(a,b,c,d),c(selectsim_only,select_only,gamtoc_only,WesME_only)) ~ 'Found in common',
                                            pairs %in% selectsim_only ~ 'SelectSim',
                                            pairs %in% select_only ~ 'Select',
                                                          pairs %in% gamtoc_only ~ 'GamTOC',
                                                         pairs %in% WesME_only ~ 'weSME/CO',
                                           TRUE ~ ''))

In [71]:
head(all_plot_df)

,pairs,effect_size,category,algo,alog_cat
,<chr>,<dbl>,<chr>,<chr>,<chr>
1,ARID1A - CARD11,0.6380518979,log,Select,Found in common
2,KDR - SLFN11,1.4144200841,log,Select,Found in common
3,BRCA1 - KDR,0.8699578453,log,Select,Found in common
4,PBRM1 - SETBP1,1.0518663974,log,Select,Found in common
5,ATRX - VAV1,1.6653570605,log,Select,Found in common
6,KRAS - STK11,0.0003633509,log,SelectSim,Found in common


In [72]:
df <- all_plot_df %>% select(pairs,effect_size,algo) %>% distinct()

In [73]:
head(df)

,pairs,effect_size,algo
,<chr>,<dbl>,<chr>
1,ARID1A - CARD11,0.6380518979,Select
2,KDR - SLFN11,1.4144200841,Select
3,BRCA1 - KDR,0.8699578453,Select
4,PBRM1 - SETBP1,1.0518663974,Select
5,ATRX - VAV1,1.6653570605,Select
6,KRAS - STK11,0.0003633509,SelectSim


In [74]:
# Calculate mean values for each group
mean_values <- aggregate(effect_size ~ algo, data = df, FUN = mean)

In [76]:
mean_values

algo,effect_size
<chr>,<dbl>
cooccur,0.53552517
GamToc,0.20895041
Select,1.12793066
SelectSim,0.08091732
WesME/CO,1.00340459


In [77]:
custom_order<-(mean_values %>% arrange((effect_size)))$algo

In [78]:
df$algo <- factor(df$algo, levels = custom_order)

In [81]:
saveRDS(df,file='/mnt/ndata/arvind/co_mutation_project/analysis/robustness_benchmarking/figures/luad_mutational_load_plot_data.rds')